In [ ]:
# 🚗 Dynamic Pricing for Urban Parking Lots – Summer Analytics 2025

# Install necessary packages
!pip install pathway bokeh panel --quiet

# Imports
import numpy as np
import pandas as pd
import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
pn.extension()

# Load dataset
from google.colab import files
uploaded = files.upload()

df = pd.read_csv("parking_stream (3).csv")  # make sure it has all required columns
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.sort_values("Timestamp").reset_index(drop=True)

# Define Schema
class ParkingSchema(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str
    Latitude: float
    Longitude: float
    LotID: str

# Load streaming data
data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)
fmt = "%Y-%m-%d %H:%M:%S"
data = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# ================================
# 🔵 MODEL 1: Baseline Linear Model
# ================================
alpha = 2.0
data_model1 = data.with_columns(
    price = 10 + alpha * (pw.this.Occupancy / pw.this.Capacity)
).with_columns(
    price = pw.this.price.apply(lambda x: round(min(max(x, 5), 20), 2))
)

model1_window = data_model1.windowby(
    pw.this.t, instance=pw.this.day,
    window=pw.temporal.tumbling(datetime.timedelta(days=1)),
    behavior=pw.temporal.exactly_once_behavior()
).reduce(
    t = pw.this._pw_window_end,
    avg_price = pw.reducers.mean(pw.this.price)
)

def plot_model1(source):
    fig = bokeh.plotting.figure(title="Model 1: Baseline Linear Pricing", x_axis_type="datetime", height=400, width=800)
    fig.line("t", "avg_price", source=source, line_width=2, color="blue")
    fig.circle("t", "avg_price", source=source, size=6, color="black")
    return fig

viz1 = model1_window.plot(plot_model1, sorting_col="t")
pn.Column(viz1).servable()

# ================================
# 🟢 MODEL 2: Demand-Based Pricing
# ================================
data2 = data.with_columns(
    traffic_score = pw.case(
        (pw.this.TrafficConditionNearby == "low", 0.0),
        (pw.this.TrafficConditionNearby == "medium", 0.5),
        (pw.this.TrafficConditionNearby == "high", 1.0),
        default=0.5
    ),
    vehicle_weight = pw.case(
        (pw.this.VehicleType == "bike", 0.5),
        (pw.this.VehicleType == "car", 1.0),
        (pw.this.VehicleType == "truck", 1.5),
        default=1.0
    )
)

alpha, beta, gamma, delta, epsilon = 1.0, 0.3, 0.5, 0.2, 0.2
data2 = data2.with_columns(
    demand = (
        alpha * (pw.this.Occupancy / pw.this.Capacity) +
        beta * pw.this.QueueLength -
        gamma * pw.this.traffic_score +
        delta * pw.this.IsSpecialDay +
        epsilon * pw.this.vehicle_weight
    ),
    demand_norm = pw.this.demand.apply(lambda x: max(0, min(np.tanh(x / 5), 1.0))),
    price = pw.this.demand_norm * 10 + 10
).with_columns(
    price = pw.this.price.apply(lambda x: round(min(max(x, 5), 20), 2))
)

model2_window = data2.windowby(
    pw.this.t, instance=pw.this.day,
    window=pw.temporal.tumbling(datetime.timedelta(days=1)),
    behavior=pw.temporal.exactly_once_behavior()
).reduce(
    t = pw.this._pw_window_end,
    avg_price = pw.reducers.mean(pw.this.price)
)

def plot_model2(source):
    fig = bokeh.plotting.figure(title="Model 2: Demand-Based Pricing", x_axis_type="datetime", height=400, width=800)
    fig.line("t", "avg_price", source=source, line_width=2, color="green")
    fig.circle("t", "avg_price", source=source, size=6, color="orange")
    return fig

viz2 = model2_window.plot(plot_model2, sorting_col="t")
pn.Column(viz2).servable()

# ================================
# 🔴 MODEL 3: Competitive Pricing
# ================================

# Haversine function
def haversine(lat1, lon1, lat2, lon2):
    from math import radians, cos, sin, asin, sqrt
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    return 6371 * 2 * asin(sqrt(a))

data_geo = data2.with_columns(base_price = pw.this.price)

joined = data_geo.join(data_geo, how="cross").filter(
    pw.this.LotID != pw.that.LotID,
    pw.this.t == pw.that.t
)

neighbors = joined.with_columns(
    dist = pw.apply(haversine, pw.this.Latitude, pw.this.Longitude, pw.that.Latitude, pw.that.Longitude),
    neighbor_price = pw.that.base_price
).filter(
    pw.this.dist <= 1.0
).groupby(pw.this.LotID, pw.this.t).reduce(
    avg_neighbor_price = pw.reducers.mean(pw.this.neighbor_price)
)

data3 = data_geo.join(neighbors, on=(pw.this.LotID == pw.that.LotID, pw.this.t == pw.that.t), how="left").with_columns(
    final_price = pw.this.base_price + pw.this.avg_neighbor_price.fillna(10).apply(lambda x: (x - 10) * 0.3)
).with_columns(
    final_price = pw.this.final_price.apply(lambda x: round(min(max(x, 5), 25), 2))
)

model3_window = data3.windowby(
    pw.this.t, instance=pw.this.day,
    window=pw.temporal.tumbling(datetime.timedelta(days=1)),
    behavior=pw.temporal.exactly_once_behavior()
).reduce(
    t = pw.this._pw_window_end,
    avg_price = pw.reducers.mean(pw.this.final_price)
)

def plot_model3(source):
    fig = bokeh.plotting.figure(title="Model 3: Competitive Pricing", x_axis_type="datetime", height=400, width=800)
    fig.line("t", "avg_price", source=source, line_width=2, color="red")
    fig.circle("t", "avg_price", source=source, size=6, color="black")
    return fig

viz3 = model3_window.plot(plot_model3, sorting_col="t")
pn.Column(viz3).servable()

# ----------------------------------------
# To execute stream processing:
# Uncomment the line below to start
# %%capture --no-display
pw.run()


Saving parking_stream.csv to parking_stream.csv


AttributeError: 'ColumnReference' object has no attribute 'apply'